# LSTM PROJECT TITLE

## PROJECT SUMMARY
Predicting future oil production rates for 5 oil wells

Using the contents of the entire workbook create a 2 paragraph project introduction for ... project. The first paragraph should describe what the project is about and the base LSTM model being used. The second paragraph gives a brief summary of the steps that will be taken to analyze, train, and evaluate the model including EDA, training, testing, hyperparameter tuning, and comparing the base model to the .......... models.

## DATA SOURCE & IMPORT

#### Data Source

- The datasets used in this project are a combination of production data for five crude oil wells and commodity pricing data.
- **Production Data**
  - The historical production data has been provided directly by a small Canadian oil and gas producer.  
  - *Reference*: Private Operator (Anonymous). (2025). Internal oil well production dataset: Five wells in Western Canada.
- **Commodity Pricing Data**
  - Historical crude oil and natural gas commodity pricing data is downloaded from a trusted Canadian petroleum reserve evaluator.
  - *Reference*: McDaniel & Associates Consultants Ltd. (2025, October 1). Price forecasts [Web page]. https://mcdan.com/price-forecasts/

#### Import Python Libraries

In [ ]:
# Import core analysis, visualization, and statistical libraries used throughout the notebook
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from pathlib import Path
import random
import itertools
import json
import requests
import warnings
import logging
from IPython.display import display

import pandas as pd
import numpy as np

from io import StringIO
from textwrap import fill
from fake_useragent import UserAgent

import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D
import tensorflow as tf
from tensorflow import keras
import optuna
from optuna.visualization import plot_parallel_coordinate

# Silence warnings
alt.data_transformers.disable_max_rows()
warnings.filterwarnings('ignore', category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)
tf.get_logger().setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

# Set seeds
seed = 666
np.random.seed(seed)
random.seed(seed)
keras.utils.set_random_seed(seed)

# Define model constants
SCALING_FACTOR = 1e-6 # Log transform scaling factor to ensure zero values are handled correctly
SPLIT_RATIO = 0.8     # Train-test or train-validation split ratio
EPOCHS = 100          # Number of epochs to train models over
BATCH_SIZE = 1        # Standard LSTM batch size
PATIENCE = 10         # Early stopping level of patience
N_TRIALS = 20         # Number of hyperparameter tuning trails to conduct

#### Import Production Data
- Production data has been provided directly from the oil company in a `.csv` file.

In [ ]:
# Import production dataset file (as provided by the private oil company)
prod_df = pd.read_csv('prod.csv')

# Convert date column to datetime
prod_df['date'] = pd.to_datetime(prod_df['date'])

# Display production dataframe
prod_df

#### Import Commodity Pricing Data
- Commodity price data in `json` format is extracted directly from the McDaniel's website.

In [ ]:
# Function to load the commodity price json file and convert it into a dataframe
def import_price_data(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().random})

    data = json.loads(response.content)

    rows = []
    for entry in data:
        date = entry['date']
        row = {'date': date}
        for item in entry['values']:
            base_key = item['key'].replace(' Forecast', '').strip()
            row[f'{base_key}'] = item['usd']
        rows.append(row)
    df = pd.DataFrame(rows)
    df['date'] = pd.to_datetime(df['date'])
    return df

# Define the url links to the pricing data
oil_url = 'https://mcdan.com/forecast/data/oil.json'
gas_url = 'https://mcdan.com/forecast/data/gas.json'

# Create oil and natural gas pricing datasets
oil_prices_df = import_price_data(oil_url)
gas_prices_df = import_price_data(gas_url)

# Define prod_df min/max dates
prod_min_date = prod_df['date'].min()
prod_max_date = prod_df['date'].max()

# Filter oil_prices_df and gas_prices_df by prod_min_date and prod_max_date
oil_prices_df = oil_prices_df[(oil_prices_df['date']>=prod_min_date) & (oil_prices_df['date']<=prod_max_date)]
gas_prices_df = gas_prices_df[(gas_prices_df['date']>=prod_min_date) & (gas_prices_df['date']<=prod_max_date)]

#### Merge Production & Pricing Datasets
- Create the final dataset used for evaluation

In [ ]:
# Merge the prod_df, oil_prices_df, and gas_prices_df
df = prod_df.merge(oil_prices_df).merge(gas_prices_df)

# Display combined dataset
df

## DATA SUMMARY

#### Dataset Shape & Data Types
- Investigating the number of rows and columns contained in the dataset, and the types of data found in each column.

In [ ]:
# Dataset shape
(rows, cols) = df.shape
print(f'The dataset has {rows:,} rows and {cols} columns.\n')

# Dataset data types
df.info()

#### Feature Descriptions
- The dataset contains a combination of production rates and commodity pricing streams.

Feature | Description | Units | Data Type
---|---|---|---
**well_name** | Name of the producing well | | Categorical
**date** | First day of each month of production | dd/mm/yyy | Datetime
**production_hours** | Number of producing hours a well is on production in a month | hours | Numeric
**oil_producing_daily_avg (bbl/day)** | Monthly average of the daily oil production rate | bbl/d | Numeric
**gas_producing_daily_avg (mcf/day)** | Monthly average of the daily natural gas production rate | mcf/d | Numeric
**water_producing_daily_avg (bbl/day)** | Monthly average of the daily water production rate | bbl/d | Numeric
**WTI** | Monthly average of the West Texas Intermediate crude oil benchmark price | US $/bbl | Numeric
**BRENT** | Monthly average of the Brent crude oil benchmark price | US $/bbl | Numeric
**EDM** | Monthly average of the Edmonton Light Sweet crude oil benchmark price | US $/bbl | Numeric
**WCS** | Monthly average of the Western Canadian Select crude oil benchmark price | US $/bbl | Numeric
**HH** | Monthly average of the Henry Hub natural gas benchmark price | US $/mcf | Numeric
**AECO** | Monthly average of the AECO natural gas benchmark price | US $/mcf | Numeric
**STA2** | Monthly average of the BC Station 2 natural gas benchmark price | US $/mcf | Numeric

> - For this project **the target variable is the `oil_producing_daily_avg (bbl/day)`** column at time $t+1$.

## EDA
The EDA section presents monthly production trends by well, blends in macro indicators, and highlights shut-in periods to reveal The EDA section traces monthly production trends for every well, overlaying macro pricing indicators and shut-in periods to surface operational cycles, sudden changes, and downtime. It also frames the upcoming train/test split within that historical context, so the LSTM preparation later clearly ties back to the exploratory insights.

#### Frequency of Categorical Columns
- Each well has produced for a different length of time, where each row in the dataset represents a month of production.
- It would be interesting to visualize the difference between the first and last day of production by counting the row occurrences in the `well_name` column.

In [ ]:
# Visualize months on production
alt.Chart(df).mark_bar().encode(
    alt.X('well_name:N').title(None),
    alt.Y('count():Q').title(None),
    alt.Color('well_name').scale(scheme='viridis')
).properties(
    height = 300,
    width = 500,
    title=alt.Title('Months on Production by Well', fontSize=20, align='right'),
)

#### Distributions of Numeric Columns
- These are used to visualize how numeric columns are distributed, and to observe any patterns or outliers in the dataset.

In [ ]:
# Function to plot a set of histograms for a subset of columns
def plot_numeric_histograms(input_df, subset, chart_title, col_count):
    # Melt the numeric columns into one column
    df_melt = input_df[subset].melt(var_name='feature', value_name='value')

    # Create a base Altair histogram chart
    base_chart = alt.Chart(df_melt).mark_bar(opacity=1, binSpacing=0).encode(
        alt.X('value:Q').axis(title=None).bin(maxbins=50),
        alt.Y('count():Q').axis(title=None).stack(None),
        color = alt.Color('feature:N').scale(scheme='viridis').sort(df.columns).legend(None)
    ).properties(
        width=750 / col_count,
        height=150
    )

    # Display a histogram for each numeric_columns
    chart = alt.ConcatChart(
        title=alt.Title(f'{chart_title}', fontSize=20),
        concat=[base_chart.transform_filter(alt.datum.feature == value).properties(title=value) for value in subset],
        columns=col_count,
    ).configure_title(
        fontSize=10
    ).resolve_axis(
        x='independent',
        y='independent'
    ).resolve_scale(
        x='independent', 
        y='independent'
    )

    return chart

# Define numeric feature columns
numeric_columns = [col for col in df.columns.to_list() if col not in ['well_name','date']]

# Plot numeric histograms
plot_numeric_histograms(df, numeric_columns, 'Numeric Columns Data Distributions', 3)

> - The `oil_producing_daily_avg (bbl/day)` and `water_producing_daily_avg (bbl/day)` columns seem to be somewhat exponentially distributed, whereas the `gas_producing_daily_avg (mcf/day)` column has more variation and looks to be a skewed distribution.
> - The crude oil commodity prices (`WTI`, `BRENT`, `EDM`, `WCS`) all seem to have bi-modal distributions indicating that those prices may experience periods of high and low price environments.

#### Numeric Columns Pairplot
- This pairplot visualizes the scatterplots and histograms that reveals how production metrics and pricing indicators move together across wells

In [ ]:
# Filter to only numeric features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
pairgrid_df = df[numeric_cols].dropna()

# Set theme
sns.set_style('whitegrid')
sns.set_context('talk', font_scale=0.9)
sns.set_palette('viridis')
viridis_colors = sns.color_palette('viridis_r', 5)
hist_color = viridis_colors[2]
scatter_color = viridis_colors[-1]
kde_cmap = sns.color_palette('viridis', as_cmap=True)

# Build the PairGrid with histograms on the diagonal, scatterplots above, and KDE contours below
pairgrid = sns.PairGrid(pairgrid_df, diag_sharey=False)
pairgrid.map_diag(sns.histplot, color=hist_color, edgecolor='white', linewidth=1.0)
pairgrid.map_upper(sns.scatterplot, color=scatter_color, s=25, alpha=0.7, edgecolor='white', linewidth=0.3)
pairgrid.map_lower(sns.kdeplot, fill=True, thresh=0.04, levels=5, cmap=kde_cmap, linewidths=0.8)

# Rotate and wrap long axis labels
for ax in pairgrid.axes.flatten():
    ax.tick_params(axis='x', labelrotation=90)
    ax.tick_params(axis='y', labelrotation=0)
    ax.set_xlabel(fill(ax.get_xlabel(), 12))
    ax.set_ylabel(fill(ax.get_ylabel(), 12))

# Remove extra whitespace and add an overall title
plt.subplots_adjust(wspace=0.05, hspace=0.05)
pairgrid.fig.suptitle('Numeric Feature Relationships', y=1.02)

pairgrid

> - Obvious correlations are observed between the production volumes and production rates. This makes sense since they are just a transformation of each other (as described in the Feature Descriptions section).
> - Correlations also exist among the crude oil pricing columns (`WTI`, `BRENT`, `EDM`, `WCS`). These are not unusual since most crude oils are priced relative to WTI. Similarly correlations are also observed for the natural gas prices between `AECO` and `STA2`. This is expected since both commodites are traded into similar markets.
> - No other obvious correlations are observed among the other numeric columns.

#### Correlation Matrix
- The correlation matrix is a heatmap of pairwise correlations between production and pricing features, helping this analysis quickly spot which variables move together before selecting inputs for the LSTM

In [ ]:
# Compute the correlation matrix across all numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df[numeric_cols].corr()

# Plot heatmap
sns.set_style('whitegrid')
sns.set_context('talk', font_scale=0.9)
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(
    corr_matrix,
    cmap='viridis',
    vmin=-1,
    vmax=1,
    annot=True,
    fmt='.4f',
    annot_kws={'size': 10},
    linewidths=0.3,
    cbar_kws={'label': 'Correlation coefficient', 'shrink': 0.8},
    square=True
)

# Resize the legend
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=10)
cbar.set_label('Correlation coefficient', size=8)

# Realign axis labels
heatmap.set_title('Numeric Feature Correlation Matrix', pad=20)
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=90, ha='right', fontsize=8)
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0, fontsize=8)
plt.tight_layout()

heatmap

> Correlations that were visually observed in the pairplot are confirmed in the correlation heatmap.

#### Production Time Series Plots
- Oil and natural gas production rates typically exponentially decline over time so they are plotted on a semi-log plot over time.

In [ ]:
# Create labels for plot
prod_rate_dict = {
    'production_hours':'Hours On Production (hrs)',
    'oil_producing_daily_avg (bbl/day)':'Oil Production (bbl/d)',
    'gas_producing_daily_avg (mcf/day)':'Gas Production (mcf/d)',
    'water_producing_daily_avg (bbl/day)':'Water Production (bbl/d)'
}

# Convert data into a dataframe for Altair plotting
prod_rate_chart_df = (
    df[['well_name', 'date'] + [k for k in prod_rate_dict.keys()]]
        .rename(prod_rate_dict, axis=1)
        .replace({0:np.nan})
        .melt(id_vars=['well_name', 'date'], value_vars=[v for v in prod_rate_dict.values()])
)

# Create plot of log-scaled time series for oil rate, gas rate, water rate, and hours on for each well
alt.Chart(prod_rate_chart_df).mark_line().encode(
    alt.X('date:T').title(None),
    alt.Y('value:Q').scale(type='log').title('Production Rate (bb/d or mcf/d)'),
    alt.Color('variable:N').scale(domain=[v for v in prod_rate_dict.values()], range=['grey', '#2ca02c', '#d62728', 'steelblue']),
    alt.Row('well_name:N').title(None),
    alt.Tooltip(['value:Q'])
).properties(
    height = 200,
    width = 800,
    title=alt.Title('Oil, Gas, and Water Production Rates Over Time', fontSize=20, anchor='start')
)

> Comments about production plots...

#### Oil Production Rate Moving Averages
- Visualizing the 12 month, 24, and 36 months oil production rate moving averages.

In [ ]:
# Create dataset for moving-average smoothing
ma_chart_df = df[['well_name', 'date', 'oil_producing_daily_avg (bbl/day)']].replace({0:np.nan})

# Define the moving-average windows
window_specs = [
    (12, '12-month MA'),
    (24, '24-month MA'),
    (36, '36-month MA'),
]

# Define moving-average line colors
window_colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
color_scale = alt.Scale(
    domain=[label for _, label in window_specs],
    range=window_colors
)

# Build base MA chart
base = alt.Chart(ma_chart_df).encode(
    alt.X('date:T').title('Date')
)

# Plot oil production line chart
raw_line = base.mark_line(color='green', opacity=0.2, size=5).encode(
    alt.Y('oil_producing_daily_avg (bbl/day):Q').title('Oil producing daily avg (bbl/day)').scale(type='log')
)

# Create each moving-average line charts
ma_layers = []
for window, label in window_specs:
    ma_layer = base.transform_window(
        rolling_mean='mean(oil_producing_daily_avg (bbl/day))',
        frame=[-(window - 1), 0],
        groupby=['well_name'],
        sort=[alt.SortField('date')]
    ).transform_calculate(
        window_label=f"'{label}'"
    ).mark_line(size=3, opacity=0.75, strokeDash=[10,4]).encode(
        y='rolling_mean:Q',
        color=alt.Color('window_label:N').scale(color_scale).title('Moving Average')
    ).properties(
        height = 200,
        width = 800,
    )
    ma_layers.append(ma_layer)

# Layer the raw_line and ma_layer charts, facet by well
layered_ma_chart = alt.layer(*ma_layers, raw_line).facet(
    row=alt.Row('well_name:N', title='Well')
).properties(
    title=alt.Title('12, 24, and 36 Month Moving Averages of the Oil Production Rate (bbl/d)', fontSize=20, anchor='start')
)

layered_ma_chart

> The 12, 24, and 36-month moving average curves show that every well’s oil production rate follows a steady downward slope, with the longer windows smoothing out the month-to-month volatility. Large negative spikes mark occasional shut-ins or outages, but the multi-year trend remains a gradual decline across all wells.

#### Oil Production Percent Change
- The month-to-month percent change chart tracks how each well’s production jumps or drops from one month to the next, helping the analysis identify volatility bursts or shut-in events to account for when modeling the LSTM

In [ ]:
# Compute month-over-month percent change in oil production for each well
pct_change_df = df[['well_name', 'date', 'oil_producing_daily_avg (bbl/day)']].copy()
pct_change_df = pct_change_df.sort_values(['well_name', 'date'])
pct_change_df['pct_change'] = pct_change_df.groupby('well_name')['oil_producing_daily_avg (bbl/day)'].pct_change() * 100
pct_change_df = pct_change_df.dropna(subset=['pct_change'])
pct_change_df['pct_change'] = pct_change_df['pct_change'].clip(-100, 100)

# Define positive/negative color scale
color_scale = alt.Scale(domain=['Increase', 'Decrease'], range=['#2ca02c', '#d62728'])

# Plot a the percent change chart
pct_change_chart = alt.Chart(pct_change_df).mark_bar().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('pct_change:Q', title='Month-to-month % change', scale=alt.Scale(domain=[-100, 100])),
    color=alt.Color(
        'change_direction:N',
        scale=color_scale,
        legend=alt.Legend(title='Direction')
    ),
    tooltip=[
        alt.Tooltip('well_name:N', title='Well'),
        alt.Tooltip('date:T', title='Date'),
        alt.Tooltip('pct_change:Q', title='% change', format='.2f')
    ]
).transform_calculate(
    change_direction="datum.pct_change >= 0 ? 'Increase' : 'Decrease'"
).properties(
    height=200,
    width=800
).facet(
    row=alt.Row('well_name:N', title='Well')
).resolve_scale(
    y='shared'
).properties(
    title=alt.Title('Month-to-Month Percent Change of the Oil Production Rate', fontSize=20, anchor='start', subtitle='(capped at +/-100%)')
)

pct_change_chart

> edit this....
> - More negative rather than positive percent changes are prevalent for each well. This makes sense because the production rate typically consistently declines over time.
> - It would be interesting to study the green bars to understand what occurred to cause an increase in oil production (which is a good thing).

#### Seasonal Decomposition Plots
- Seasonal decomposition charts break each time series into trend, seasonal, and residual components, revealing long-term decline, recurring seasonal patterns, and irregular shocks separately.
- In this analysis they help determine whether seasonal patterns or residual noise should be modeled explicitly or if the LSTM can focus mostly on the trend and notable anomalies.

In [ ]:
# Collect seasonal decomposition components for every well
decomposition_records = []
value_col = 'oil_producing_daily_avg (bbl/day)'

for well_name, group in df[['well_name', 'date', value_col]].dropna(subset=[value_col]).sort_values('date').groupby('well_name'):
    well_series = group.set_index('date')[value_col]
    monthly_series = well_series.resample('MS').mean().interpolate(limit_direction='both')

    if len(monthly_series.dropna()) < 24:
        continue

    decomposition = seasonal_decompose(
        monthly_series,
        model='additive',
        period=12,
        extrapolate_trend='freq'
    )

    component_map = {
        'Observed': decomposition.observed,
        'Trend': decomposition.trend,
        'Seasonal': decomposition.seasonal,
        'Residual': decomposition.resid
    }

    for component_name, component_values in component_map.items():
        component_df = component_values.reset_index()
        component_df.columns = ['date', 'value']
        component_df['component'] = component_name
        component_df['well_name'] = well_name
        decomposition_records.append(component_df)

# Combine component data and drop rows created by leading/trailing NaNs
seasonal_decomposition_df = pd.concat(decomposition_records, ignore_index=True)
seasonal_decomposition_df = seasonal_decomposition_df.dropna(subset=['value'])
component_order = ['Observed', 'Trend', 'Seasonal', 'Residual']

# Plot the observed, trend, seasonal, and residual series for each well in a faceted grid
seasonal_decomposition_chart = alt.Chart(seasonal_decomposition_df).mark_line(color='#2ca02c').encode(
    x=alt.X('date:T').title(None),
    y=alt.Y('value:Q').title(None),
    row=alt.Row('component:N', sort=component_order, title=None),
    column=alt.Column('well_name:N', title='Well'),
    color=alt.Color('well_name:N', title='Well', scale=alt.Scale(scheme='viridis'))
).resolve_scale(
    y='independent'
).properties(
    height=100,
    width=200,
    title=alt.Title('Seasonal Decomposition of the Oil Production Rate', fontSize=20, anchor='start')
)

seasonal_decomposition_chart

> The seasonal decomposition charts show a steep decline in each well’s long-term trend followed by a flat tail, while the seasonal component oscillates annually with consistent amplitude—**clear evidence of recurring 12-month seasonality**. Residuals remain relatively small after the initial drop, suggesting most structure is captured by the trend and seasonal terms. LSTM's inherently handle seasonality, but if model fits are insufficient then we could consider applying temporal differencing to account for these cyclical effects.

#### Autocorrelation Plot
- The autocorrelation plot shows how each well’s production correlates with its past values at different lags, helping this analysis gauge how far back the LSTM needs to look to capture meaningful temporal dependencies.

In [ ]:
# Group by well
well_groups = (
    df[['well_name', 'date', 'oil_producing_daily_avg (bbl/day)']]
    .dropna(subset=['oil_producing_daily_avg (bbl/day)'])
    .sort_values('date')
    .groupby('well_name')
)

# Define horizon to analyze autocorrelation over
horizon = 36

# Create ACF datasets
acf_records = []
for well_name, group in well_groups:
    monthly_series = group.set_index('date')[value_col].resample('MS').mean().interpolate(limit_direction='both')
    acf_values = acf(monthly_series, nlags=horizon, fft=False, missing='drop')
    for lag in range(1, horizon + 1):
        acf_records.append({
            'well_name':     well_name,
            'lag':           lag,
            'acf':           acf_values[lag],
            'horizon_label': f"{horizon}-month"
        })

# Plot ACF chart
acf_df = pd.DataFrame(acf_records)
acf_chart = alt.Chart(acf_df).mark_bar().encode(
    x=alt.X('lag:O', title='Lag (months)', axis=alt.Axis(values=[6, 12, 18, 24, 30, 36])),
    y=alt.Y('acf:Q', title='Autocorrelation', scale=alt.Scale(domain=[-1, 1])),
    column=alt.Column('well_name:N', title=None),
    row=alt.Row('horizon_label:N', sort=[f"36-month"], title='Horizon'),
    color=alt.condition(alt.datum.acf >= 0, alt.value('#1f77b4'), alt.value('#d62728'))
).resolve_scale(
    y='shared'
).properties(
    height=200,
    width=200,
    title=alt.Title('36 month Autocorrelation of the Oil Production Rate', fontSize=20, anchor='start')
)

acf_chart

> Autocorrelation stays strongly positive through roughly the first 12–18 months for every well, confirming that recent production levels heavily influence the next year’s output. Beyond about 24 months the correlations decay to near zero or slightly negative, suggesting the model can safely prioritize shorter lookback windows without losing much signal. With this knowledge **I'll consider testing look back windows of 12, 24, and 36 months for the LSTM models**.

## DATA CLEANING & PREPROCESSING

#### Adding Columns Required for Modeling
- A `months_on` column is added that calculates the number of months the well has been on production since its first month of production.
- A set of dummy variable columns are created that define general states of the oil and gas industry over time.

In [ ]:
# Add a months_on column which is the number of months a well has produced since it was first put on production
df['months_on'] = df.groupby('well_name')['date'].transform(lambda s: s.rank(method='first').astype(int))

In [ ]:
# Add columns that identify key events affecting the oil and gas industry
indicator_windows = {
    'middle_instability_tightening_supply': [('2009-01-01', '2014-08-01')],
    'opec_oversupply_during_downturn':      [('2014-09-01', '2016-02-01')],
    'opec_undersupply_during_recovery':     [('2016-03-01', '2020-02-01')],
    'covid':                                [('2020-03-01', '2022-06-01')],
    'ukraine_war':                          [('2022-02-01', '2025-11-01')]
}

# Create date index
min_prod_date = df['date'].min().to_period('M').to_timestamp()
date_index = pd.date_range(start=min_prod_date, end='2025-11-01', freq='MS')

# Create & populate dummy variables
dummy_df = pd.DataFrame({'date': date_index})
for indicator, windows in indicator_windows.items():
    dummy_df[indicator] = 0
    for start, end in windows:
        start_ts = pd.Timestamp(start)
        end_ts = pd.Timestamp(end)
        mask = (dummy_df['date'] >= start_ts) & (dummy_df['date'] <= end_ts)
        dummy_df.loc[mask, indicator] = 1

# Merge dummy_df with the dataset
df = df.merge(dummy_df, how='left')

# Display the updated dataset
df

#### Scale Data
- Models are trained using the concatenation of all the well's `X_train` and `y_train` datasets, and then individually predicted and tested using each wells `X_test` and `y_test` datasets.
- Because of this I scale the entire dataset's numeric features to a mean of 0 and standard deviation of 1 to improve generality of the LSTM models.
- Numeric features that are scaled are:
  - `production_hours`
  - `oil_producing_daily_avg (bbl/day)`
  - `gas_producing_daily_avg (mcf/day)`
  - `water_producing_daily_avg (bbl/day)`
  - `WTI`
  - `BRENT`
  - `EDM`
  - `WCS`
  - `HH`
  - `AECO`
  - `STA2`
  - `months_on`

In [ ]:
# Fit the StandardScaler so X features can be scaled later
scaler = StandardScaler()
scaler.fit(df.iloc[:, 2:14])

#### Train-Test Split
- LSTM models expect a 3-dimensional dataframe as input with a shape of: ($\textit{samples}$, $\textit{look back}$, $\textit{number of features}$).
  - Where:
    - $\textit{samples}$ = are the number of unique time slices generated from the set of features $X$
    - $\textit{look back}$ = is the length of each sample (measured in months)
    - $\textit{number of features}$ = are the number of feature variables in the input matrix $X$
- The `create_dataset` function below creates the 3-dimensional LSTM inputs for a given input dataset
  - The target values $y$ are the `oil_producing_daily_avg (bbl/day)` values taken at time $t+1$
  - The feature matrix $X$ contains 17 features:
    - `production_hours`
    - `oil_producing_daily_avg (bbl/day)`
    - `gas_producing_daily_avg (mcf/day)`
    - `water_producing_daily_avg (bbl/day)`
    - `WTI`
    - `BRENT`
    - `EDM`
    - `WCS`
    - `HH`
    - `AECO`
    - `STA2`
    - `months_on`
    - `middle_instability_tightening_supply`
    - `opec_oversupply_during_downturn`
    - `opec_undersupply_during_recovery`
    - `covid`
    - `ukraine_war`
- $X$ and $y$ are split into training and testing sets, where:
  - the first 80% of the time series is used as the training set, and 
  - the last 20% of the time series is used as the testing set
- Scaling is applied where:
  - The $X$ numeric values are scaled using the standard scaler mentioned above
  - $y$ is log transformed (with a scaling factor of 1e-6 added to avoid errors when $y=0$)
- The `X_train` and `y_train` sets for each well are stacked into one so the model can be trained on one complete training dataset

In [ ]:
# Function to scale and split a dataset into X_train, X_test, y_train, y_test
def create_train_test_split(dataset, look_back, scaler):
    # Drop date and well_name columns
    dataset = dataset[[col for col in dataset.columns if col not in ['date', 'well_name']]]

    # Split into X & y
    X = dataset.iloc[0:(len(dataset)-1), :].to_numpy()
    y = dataset.iloc[1:, 1]

    # Scale the numeric features in X
    X[:,:12] = scaler.transform(X[:,:12])

    # Log transform y
    y_log = np.log(y + SCALING_FACTOR).to_numpy()

    # Split into train & test sets
    train_idx = int(len(X) * SPLIT_RATIO)
    X_train, X_test, y_train, y_test = [], [], [], []

    # Create X_train & y_train datasets
    for i in range(train_idx - look_back + 1):
        X_train_sample = X[i:(i + look_back), :]
        X_train.append(X_train_sample)
        y_train.append(y_log[i + look_back - 1])
    X_train = np.array(X_train) # X_train.shape = (train_idx - look_back + 1, look_back, n_features)
    y_train = np.array(y_train) # y_train.shape = (train_idx - look_back + 1, )

    # Create X_test & y_test datasets
    for i in range(train_idx - look_back + 1, len(X) - look_back + 1):
        X_test_sample = X[i:(i+look_back), :]
        X_test.append(X_test_sample)
        y_test.append(y_log[i + look_back - 1])
    X_test = np.array(X_test) # X_test.shape = (len(X) - train_idx, look_back, n_features)
    y_test = np.array(y_test) # y_test.shape = (len(y) - train_idx, )

    return X_train, X_test, y_train, y_test

# Function to loop over look_back windows and well_names to create a dictionary of train-test datasets that will be used for model training
def create_train_test_dict(df, scaler):
    train_test_dict = {}
    for look_back in [12, 24, 36]:
        for well_name in df['well_name'].unique():

            # Ensure nested levels exist
            if well_name not in train_test_dict:
                train_test_dict[well_name] = {}
            if look_back not in train_test_dict[well_name]:
                train_test_dict[well_name][look_back] = {}

            # Extract dataset
            dataset = df[df['well_name'] == well_name]
            X_train, X_test, y_train, y_test = create_train_test_split(dataset, look_back, scaler)

            # Create entires
            train_test_dict[well_name][look_back] = {
                'X_train': X_train,
                'X_test':  X_test,
                'y_train': y_train,
                'y_test':  y_test
            }
            
    return train_test_dict

# Generate datasets dictionary
train_test_dict = create_train_test_dict(df, scaler)

In [ ]:
# Dataset shapes from train_test_dict for X_train and y_train for lookbacks = 12, 24, and 36 months
print('X_train has shape (samples, look_back, n_features)\ny_train has shape (samples, )')
for look_back in [12, 24, 36]:
    _x_train = np.concatenate([train_test_dict[well_name][look_back]['X_train'] for well_name in df['well_name'].unique()], axis=0)
    _y_train = np.concatenate([train_test_dict[well_name][look_back]['y_train'] for well_name in df['well_name'].unique()], axis=0)
    print(f'\nCombined training dataset shapes for look_back = {look_back}:\n-> X_train : {_x_train.shape}\n-> y_train : {_y_train.shape}')

## LSTM MODELING
Describe base vanilla LSTM model...

#### Define LSTM Models
- In this analysis I'll be evaluating 4 types of LSTM model. The first is a Vanilla LSTM, which I call my Base LSTM
- The base model is used as a datum to compare to the other models to
- The other models are the Bidirectional LSTM, Stacked LSTM, and CNN-LSTM.

In [ ]:
# Define LSTM model instances
def create_LSTM_model(model_type, n_nodes, look_back, learning_rate, dropout, n_features):
    strategy = tf.distribute.MirroredStrategy() # Utilize GPUs
    with strategy.scope():
        model = Sequential()
        if model_type == 'base': # Base LSTM
            model.add(LSTM(n_nodes, input_shape=(look_back, n_features)))
        if model_type == 'bidirectional': # Bidirectional LSTM
            model.add(Bidirectional(LSTM(n_nodes), input_shape=(look_back, n_features)))
        if model_type == 'stacked': # Stacked LSTM
            model.add(LSTM(n_nodes * 2, return_sequences=True, input_shape=(look_back, n_features)))
            model.add(LSTM(n_nodes))
        if model_type == 'cnn': # CNN-LSTM
            model.add(Conv1D(filters=32, kernel_size=3, padding='causal', input_shape=(look_back, n_features)))
            model.add(MaxPooling1D(pool_size=2))
            model.add(LSTM(n_nodes))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=learning_rate))
    return model

#### Fit Base LSTM Model
- Fit an LSTM model using given X_train and y_train
- Function generates:
  - Train & test RMSE scores
  - Train-validation loss plot (with 20% of the data used for validation)
  - Train and test actual vs pred line plots

In [ ]:
# Fit base LSTM
def fit_lstm(model):
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE) # Add early stopping to prevent overfitting
    history = model.fit(
        X_train,
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=(1-SPLIT_RATIO),
        shuffle=False,
        verbose=0,
        callbacks=[early_stopping]
    )
    return history

# Create loss plot
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Train Loss vs Validation Loss ({well_name})')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')

# Define base LSTM model inputs
model_type = 'base'
look_back = 12
n_nodes = 4
learning_rate = 0.001
dropout = 0.1

# Generate X_train and y_train datasets for the given n_nodes, look_back and learning_rate
X_train = np.concatenate([train_test_dict[well_name][look_back]['X_train'] for well_name in train_test_dict.keys()], axis=0)
y_train = np.concatenate([train_test_dict[well_name][look_back]['y_train'] for well_name in train_test_dict.keys()], axis=0)

# Generate the base model instance and display model summary
model = create_LSTM_model(model_type, n_nodes, look_back, learning_rate, dropout, X_train.shape[2])
model.summary()

# Fit the base model LSTM and display the loss plot
history = fit_lstm(model)
plot_loss(history)

> Comments on loss plot ...

#### Well 1 Base LSTM Prediction

In [ ]:
# Function to predict oil production from train-test datasets
def predict_oil_production(train_test_dict, well_name, look_back, history):
    X_train = train_test_dict[well_name][look_back]['X_train']
    y_train = train_test_dict[well_name][look_back]['y_train']
    X_test = train_test_dict[well_name][look_back]['X_test']
    y_test = train_test_dict[well_name][look_back]['y_test']

    # Make train-test predictions
    trainPredict = history.model.predict(X_train, verbose=0)
    testPredict = history.model.predict(X_test, verbose=0)

    # Unscale target variables and predictions
    y_train = np.exp(y_train) - SCALING_FACTOR
    trainPredict = np.exp(trainPredict) - SCALING_FACTOR
    y_test = np.exp(y_test) - SCALING_FACTOR
    testPredict = np.exp(testPredict) - SCALING_FACTOR

    # Calculate train-test RMSE
    print('\n============================================================================')
    print(f'Root Mean Squared Error Scores ({well_name})')
    print('============================================================================\n')   
    train_rmse = np.sqrt(mean_squared_error(y_train, trainPredict))
    test_rmse = np.sqrt(mean_squared_error(y_test, testPredict))
    print(f'Train RMSE: {train_rmse:,.1f}')
    print(f'Test RMSE: {test_rmse:,.1f}\n')

    # Actual vs predicted train-test line charts
    print('\n============================================================================')
    print(f'Actual vs Predicted Plots ({well_name})')
    print('============================================================================\n')   
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6), sharey=True)
    ax1.plot(y_train, label='y_train Actual')
    ax1.plot(trainPredict, label='y_train Predicted')
    ax1.legend()
    ax1.set_ylabel('Oil Production Rate (bbl/d)')
    ax1.set_xlabel('Months')
    ax2.plot(y_test, label='y_test Actual')
    ax2.plot(testPredict, label='y_test Predicted')
    ax2.legend()
    ax2.set_xlabel('Months')
    fig.suptitle(f'Oil Production Rate Train/Test Actual vs Predicted ({well_name})')

# Fit LSTM model and predict for well_1
predict_oil_production(train_test_dict, 'well_1', look_back, history)

#### Well 2 Base LSTM Prediction

In [ ]:
# Fit LSTM model and predict for well_2
predict_oil_production(train_test_dict, 'well_2', look_back, history)

#### Well 3 Base LSTM Prediction

In [ ]:
# Fit LSTM model and predict for well_3
predict_oil_production(train_test_dict, 'well_3', look_back, history)

#### Well 4 Base LSTM Prediction

In [ ]:
# Fit LSTM model and predict for well_4
predict_oil_production(train_test_dict, 'well_4', look_back, history)

#### Well 5 Base LSTM Prediction

In [ ]:
# Fit LSTM model and predict for well_5
predict_oil_production(train_test_dict, 'well_5', look_back, history)

> Something about base predictions

## HYPERPARAMETER TUNING
Using the selected model parameters of `lookback = 12`, `n_nodes = 4`, `learning_rate = 0.001`, and `dropout = 0.1` captured the general trends of the training data, but the fit could be improved. Next I'll tune the parameters using the Optuna bayesian optimizer to find the ideal model for each well.

#### Create Tuning Functions
- ...

In [ ]:
# Function to evaluate a single hyperparameter tuning trial
def _evaluate_trial(model_type, trial, train_test_dict, well_name):
    # Define parameter ranges to test 
    look_back = trial.suggest_categorical('look_back', [12, 24, 36])
    n_nodes = trial.suggest_categorical('n_nodes', [4, 8, 16])
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 0.5, log=True)
    dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.1)

    # Get X_train and y_train datasets based on the look_back window
    X_train = np.concatenate([train_test_dict[well_name][look_back]['X_train'] for well_name in train_test_dict.keys()], axis=0)
    y_train = np.concatenate([train_test_dict[well_name][look_back]['y_train'] for well_name in train_test_dict.keys()], axis=0)

    # Create an LSTM model instance for the selected hyperparameters
    model = create_LSTM_model(model_type, n_nodes, look_back, learning_rate, dropout, X_train.shape[2])
    
    # Fit LSTM model
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE)
    model.fit(
        X_train,
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=(1-SPLIT_RATIO),
        shuffle=False,
        verbose=0,
        callbacks=[early_stopping],
    )

    # Define the per-well X_test and y_test datasets and predict the y_test values
    X_test = train_test_dict[well_name][look_back]['X_test']
    y_test = train_test_dict[well_name][look_back]['y_test']
    y_test = np.exp(y_test) - SCALING_FACTOR
    testPredict = model.predict(X_test, verbose=0)
    testPredict = np.exp(testPredict) - SCALING_FACTOR

    # Calculate the RMSE using the y_test and testPredict values
    test_rmse = np.sqrt(mean_squared_error(y_test, testPredict))

    # Clear keras session
    keras.backend.clear_session()

    # Summarize selected hyperparameters
    trial.set_user_attr('look_back', look_back)
    trial.set_user_attr('n_nodes', n_nodes)
    trial.set_user_attr('learning_rate', learning_rate)
    trial.set_user_attr('dropout', dropout)

    # Display the results of the current trial
    print(f'(look_back={look_back}, n_nodes={n_nodes}, learning_rate={learning_rate:.4f}, dropout={dropout:.4f}) -> RMSE={test_rmse:.4f}')

    return float(test_rmse)

# Function to apply Bayesian hyperparameter tuning using the Optuna package
def bayesian_tune_hyperparameters(model_type):
    # Check if hyperparameters have already been tuned and load from directory (used to optimize re-runs of the worksheet)
    if os.path.exists(f'tuning_rmse_results_{model_type}.csv'):
        return pd.read_csv(f'tuning_rmse_results_{model_type}.csv')
    else: 
        # If the hyperparameters have not been previously tuned then continue to tuning
        trial_records = []
        well_names = df['well_name'].unique()

        # Loop over the well_names and conduct tuning trials
        for well_name in well_names:
            print(f'{model_type} Model Bayesian optimization trials for {well_name}:')
            
            # Optuna objective function to optimize
            def objective(trial):
                return _evaluate_trial(model_type, trial, train_test_dict, well_name)
            
            # Create an Optuna study instance and run tuning trials
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True, gc_after_trial=True)

            # Save information on the best trial
            best_trial = study.best_trial
            trial_records.extend(
                [
                    {
                        'well_name': well_name,
                        'trial': trial.number,
                        'look_back': trial.params.get('look_back'),
                        'n_nodes': trial.params.get('n_nodes'),
                        'learning_rate': trial.params.get('learning_rate'),
                        'dropout': trial.params.get('dropout'),
                        'test_rmse': trial.value,
                    }
                    for trial in study.trials
                ]
            )

            # Create a parallel coordinate plot summarizing the tuning results
            parallel_fig = plot_parallel_coordinate(
                study,
                params=['look_back', 'n_nodes', 'learning_rate', 'dropout'],
                target=lambda trial: trial.value,
                target_name='RMSE'
            ).update_layout(
                title_text=f'Hyperparameter Tuning Parallel Coordinate Plot - {well_name.replace("_"," ").title()}'
            )
            display(parallel_fig)

        # Save the RMSE scores for each trial and return a dataframe of RMSE scores
        rmse_df = pd.DataFrame(trial_records)
        rmse_df.to_csv(f'tuning_rmse_results_{model_type}.csv', index=False)
        return rmse_df

# Run Optuna hyperparameter tuning for the Base LSTM model
base_rmse_df = bayesian_tune_hyperparameters(model_type)

> something about base HP tuning results...

#### Boxplot of Hyperparameter Tuning RMSE Results
- What is this...
- Extreme outliers in `base_rmse_df` are hidden using a +/- 1.5 * IQR filter

In [ ]:
# Function to filter outliers for each well using the 1.5*IQR rule
def rmse_iqr_filter(rmse_df):
    quartiles = rmse_df.groupby('well_name')['test_rmse'].quantile([0.25, 0.75]).unstack()
    quartiles.columns = ['q1', 'q3']
    quartiles['iqr'] = quartiles['q3'] - quartiles['q1']
    quartiles['lower'] = quartiles['q1'] - 1.5 * quartiles['iqr']
    quartiles['upper'] = quartiles['q3'] + 1.5 * quartiles['iqr']
    filtered_rmse_df = rmse_df.merge(quartiles[['lower', 'upper']], left_on='well_name', right_index=True)
    filtered_rmse_df = filtered_rmse_df[(filtered_rmse_df['test_rmse'] >= filtered_rmse_df['lower']) & (filtered_rmse_df['test_rmse'] <= filtered_rmse_df['upper'])]
    filtered_rmse_df = filtered_rmse_df.drop(columns=['lower', 'upper'])
    return filtered_rmse_df

# Function to create a set of boxplot to see the distribution of the RMSE scores attained in the hyperparameter tuning trials
def rmse_boxplots(rmse_df, model_type):
    # Filter rmse_df to remove outliers
    filtered_rmse_df = rmse_iqr_filter(rmse_df)
    
    # Boxplot showing RMSE variation by well
    rmse_boxplot = (
        alt.Chart(filtered_rmse_df)
        .mark_boxplot(size=130)
        .encode(
            x=alt.X('well_name:N', title='Well'),
            y=alt.Y('test_rmse:Q', title='Test RMSE'),
            color=alt.Color('well_name:N', legend=None)
        )
        .properties(
            width=800,
            height=400,
            title=alt.Title(
                f'{model_type} Model Hyperparameter Tuning RMSE Variation by Well',
                subtitle='Note: Extreme outliers have been hidden using a +/- 1.5 * IQR filter',
                fontSize=20,
                anchor='start'
            )
        )
    )
    return rmse_boxplot

# Create boxplots summarizing the Base LSTM model's RMSE tuning results
rmse_boxplots(base_rmse_df, 'Base')

> Something about base RMSE boxplots...

#### Apply Best Hyperparameters
- Hyperparameters are applied based on the best RMSE score for each well
- Since model is being refit after hyperparameter tuning the train and test RMSE values may stochastically vary from what Optuna produces

In [ ]:
# Function to fit and predict an LSTM model using the best hyperparameter tuning results for each well 
def apply_best_hyperparameters(rmse_df, train_test_dict, well_name, model_type):
    # Find the set of best hyperparameters
    best_entry = rmse_df[rmse_df['well_name'] == well_name].sort_values('test_rmse').iloc[0]

    # Extract and display the best hyperparameters
    look_back = int(best_entry['look_back'])
    n_nodes = int(best_entry['n_nodes'])
    learning_rate = best_entry['learning_rate']
    dropout = best_entry['dropout']
    print(f'Best hyperparameters for {well_name}:\n\
    look_back = {look_back}\n\
    n_nodes = {n_nodes}\n\
    learning_rate = {learning_rate:.4f}\n\
    dropout = {dropout:.4f}'
    )

    # Create X_train and y_train datasets for the given look_back window
    X_train = np.concatenate([train_test_dict[well_name][look_back]['X_train'] for well_name in train_test_dict.keys()], axis=0)
    y_train = np.concatenate([train_test_dict[well_name][look_back]['y_train'] for well_name in train_test_dict.keys()], axis=0)

    # Create and LSTM model instance for the given best hyperparameters
    model = create_LSTM_model(model_type, n_nodes, look_back, learning_rate, dropout, X_train.shape[2])

    # Fit the LSTM model and predict
    history = fit_lstm(model)
    predict_oil_production(train_test_dict, well_name, look_back, history)

# Apply best base tuned hyperparameters to Well 1
apply_best_hyperparameters(base_rmse_df, train_test_dict, 'well_1', 'base')

In [ ]:
# Apply best base tuned hyperparameters to Well 2
apply_best_hyperparameters(base_rmse_df, train_test_dict, 'well_2', 'base')

In [ ]:
# Apply best base tuned hyperparameters to Well 3
apply_best_hyperparameters(base_rmse_df, train_test_dict, 'well_3', 'base')

In [ ]:
# Apply best base tuned hyperparameters to Well 4
apply_best_hyperparameters(base_rmse_df, train_test_dict, 'well_4', 'base')

In [ ]:
# Apply best base tuned hyperparameters to Well 5
apply_best_hyperparameters(base_rmse_df, train_test_dict, 'well_5', 'base')

> Something about applying best HP to base...

## ALTERNATIVE LSTM MODELS
- Bidirectional LSTM
- Stacked LSTM
- CNN-LSTM

#### Bidirectional LSTM
- describe ...

In [ ]:
# Create tuned Bidirectional LSTM models
model_type = 'bidirectional'
bidirectional_rmse_df = bayesian_tune_hyperparameters(model_type)

> something about HP tuning results...

In [ ]:
# Display RMSE distributions
rmse_boxplots(bidirectional_rmse_df, model_type)

> something about boxplots...

In [ ]:
# Apply best Bidirectional LSTM tuned hyperparameters to Well 1
apply_best_hyperparameters(bidirectional_rmse_df, train_test_dict, 'well_1', model_type)

In [ ]:
# Apply best Bidirectional LSTM tuned hyperparameters to Well 2
apply_best_hyperparameters(bidirectional_rmse_df, train_test_dict, 'well_2', model_type)

In [ ]:
# Apply best Bidirectional LSTM tuned hyperparameters to Well 3
apply_best_hyperparameters(bidirectional_rmse_df, train_test_dict, 'well_3', model_type)

In [ ]:
# Apply best Bidirectional LSTM tuned hyperparameters to Well 4
apply_best_hyperparameters(bidirectional_rmse_df, train_test_dict, 'well_4', model_type)

In [ ]:
# Apply best Bidirectional LSTM tuned hyperparameters to Well 5
apply_best_hyperparameters(bidirectional_rmse_df, train_test_dict, 'well_5', model_type)

> Something about best HPs...

#### Stacked LSTM
- describe ...

In [ ]:
# Create tuned Stacked LSTM models
model_type = 'stacked'
stacked_rmse_df = bayesian_tune_hyperparameters(model_type)

> something about HP tuning results...

In [ ]:
# Display RMSE distributions
rmse_boxplots(stacked_rmse_df, model_type)

> something about boxplots...

In [ ]:
# Apply best Stacked LSTM tuned hyperparameters to Well 1
apply_best_hyperparameters(stacked_rmse_df, train_test_dict, 'well_1', model_type)

In [ ]:
# Apply best Stacked LSTM tuned hyperparameters to Well 2
apply_best_hyperparameters(stacked_rmse_df, train_test_dict, 'well_2', model_type)

In [ ]:
# Apply best Stacked LSTM tuned hyperparameters to Well 3
apply_best_hyperparameters(stacked_rmse_df, train_test_dict, 'well_3', model_type)

In [ ]:
# Apply best Stacked LSTM tuned hyperparameters to Well 4
apply_best_hyperparameters(stacked_rmse_df, train_test_dict, 'well_4', model_type)

In [ ]:
# Apply best Stacked LSTM tuned hyperparameters to Well 5
apply_best_hyperparameters(stacked_rmse_df, train_test_dict, 'well_5', model_type)

> Something about best HPs...

#### CNN-LSTM
- describe ...

In [ ]:
# Create tuned CNN-LSTM models
model_type = 'cnn'
cnn_rmse_df = bayesian_tune_hyperparameters(model_type)

> something about HP tuning results...

In [ ]:
# Display RMSE distributions
rmse_boxplots(cnn_rmse_df, model_type)

> something about boxplots...

In [ ]:
# Apply best CNN-LSTM tuned hyperparameters to Well 1
apply_best_hyperparameters(cnn_rmse_df, train_test_dict, 'well_1', model_type)

In [ ]:
# Apply best CNN-LSTM tuned hyperparameters to Well 2
apply_best_hyperparameters(cnn_rmse_df, train_test_dict, 'well_2', model_type)

In [ ]:
# Apply best CNN-LSTM tuned hyperparameters to Well 3
apply_best_hyperparameters(cnn_rmse_df, train_test_dict, 'well_3', model_type)

In [ ]:
# Apply best CNN-LSTM tuned hyperparameters to Well 4
apply_best_hyperparameters(cnn_rmse_df, train_test_dict, 'well_4', model_type)

In [ ]:
# Apply best CNN-LSTM tuned hyperparameters to Well 5
apply_best_hyperparameters(cnn_rmse_df, train_test_dict, 'well_5', model_type)

> Something about best HPs...

## RESULTS & ANALYSIS
- something about charting the progression of RMSE for the HP tuned models...

In [ ]:
# Define a dictionary of the RMSE dataframes for each LSTM model
model_types_dict = {
    'Base LSTM':          base_rmse_df,
    'Bidirectional LSTM': bidirectional_rmse_df,
    'Stacked LSTM':       stacked_rmse_df,
    'CNN-LSTM':           cnn_rmse_df
}

# Find the best RMSE for each well and model type
best_rmse_df = pd.DataFrame([])
for model_type, df in model_types_dict.items():
    df = df.copy()
    df['model_type'] = model_type
    df = df[['well_name', 'test_rmse', 'model_type']].groupby(['well_name', 'model_type'], as_index=False).min()
    best_rmse_df = pd.concat([best_rmse_df, df])

best_rmse_df

In [ ]:
# Plot best RMSE across models
alt.Chart(best_rmse_df).mark_line(
        size=5,
        strokeCap='round'
    ).encode(
        x=alt.X('model_type:N', title='Model Type'),
        y=alt.Y('test_rmse:Q', title='Test RMSE'),
        color=alt.Color('well_name:N', title='Well', scale=alt.Scale(scheme='viridis'))
    ).properties(
        width=600,
        height=400,
        title=alt.Title(
            'RMSE Progression by Well and Model Type',
            subtitle='Note: Best hyperparameter tuned RMSE used for each model type',
            fontSize=20,
            anchor='start'
        )
    )

> Summarize RMSE results. Highlight best models types for each well.

## CONCLUSION
Write a paragraph that summarizes:
1. Re-summarize project
2. Learning and takeaways
3. Why something didn’t work
4. Suggestions of ways to improve the model
    - Apply differencing
    - compare to other time series like ARMIA, Exponential Smoothing. or Prophet